In [30]:
!pip install torchmetrics

   ---------------------------------------- 0.0/926.4 kB ? eta -:--:--
   - ------------------------------------- 30.7/926.4 kB 660.6 kB/s eta 0:00:02
   -- ------------------------------------ 61.4/926.4 kB 812.7 kB/s eta 0:00:02
   ------- -------------------------------- 163.8/926.4 kB 1.2 MB/s eta 0:00:01
   -------------- ------------------------- 327.7/926.4 kB 1.8 MB/s eta 0:00:01
   ---------------------- ----------------- 522.2/926.4 kB 2.3 MB/s eta 0:00:01
   ------------------------------ --------- 716.8/926.4 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------  921.6/926.4 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 926.4/926.4 kB 2.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from torchmetrics import Accuracy
from torch import nn
from torch.nn import functional as F
from sklearn import preprocessing
from enum import Enum 

In [45]:
df = pd.read_csv("A_CProj/College/CSS484Assign4/pytorchML/export.csv")
df.head()

,file_name,avg_energy,zero_cross_range,bandwidth,spectral_centroid,label
0,Audio_Files/Music\mu1.wav,0.013581,0.245165,7995.117188,3442.537003,True
1,Audio_Files/Music\mu10.wav,0.006129,0.124285,7995.117188,1599.206709,True
2,Audio_Files/Music\mu11.wav,0.006399,0.061674,7995.117188,1003.077498,True
3,Audio_Files/Music\mu12.wav,0.010394,0.235691,7995.117188,2206.099911,True
4,Audio_Files/Music\mu13.wav,0.016870,0.068174,7995.117188,1352.087533,True


In [46]:
class CustomDataset(Dataset):
    def __init__(self):
        self.df=pd.read_csv("A_CProj/College/CSS484Assign4/pytorchML/export.csv")
        # extract labels
        self.df_labels=df[['label']]
        # drop non numeric columns to make tutorial simpler, in real life do categorical encoding
        self.df=df.drop(columns=['label','file_name'])
        # conver to torch dtypes
        self.dataset=torch.tensor(self.df.to_numpy()).float()

        self.labels=torch.tensor(self.df_labels.to_numpy().reshape(-1)).long()
    
    # This returns the total amount of samples in your Dataset
    def __len__(self):
        return len(self.dataset)
    
    # This returns given an index the i-th sample and label
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

In [47]:
ds = CustomDataset()
ds

In [48]:
dl=DataLoader(ds,batch_size=32, num_workers=0, shuffle=True,drop_last=True)
counter=0
for epoch in [1,2,3]:
    for batch,label in dl:
        if counter==0:
            print(batch,label)
            counter+=1
        print(batch.size(),label.size())

tensor([[3.9441e-03, 1.3558e-01, 7.9951e+03, 2.3474e+03],
        [3.4100e-03, 1.1585e-01, 7.9980e+03, 2.9477e+03],
        [1.5836e-02, 7.9609e-02, 7.9980e+03, 8.8044e+02],
        [1.3581e-02, 2.4516e-01, 7.9951e+03, 3.4425e+03],
        [3.6661e-02, 3.9939e-02, 7.9951e+03, 1.5273e+03],
        [5.5412e-04, 1.3512e-01, 7.9980e+03, 2.5808e+03],
        [5.0236e-04, 1.3142e-01, 7.9980e+03, 1.6306e+03],
        [7.1045e-03, 1.7277e-01, 7.9980e+03, 1.9264e+03],
        [1.5031e-01, 1.0316e-01, 7.9980e+03, 7.8838e+02],
        [7.6719e-02, 9.8720e-02, 7.9951e+03, 2.4326e+03],
        [3.1784e-04, 5.7896e-02, 7.9980e+03, 9.0200e+02],
        [4.5429e-04, 1.2189e-01, 7.9980e+03, 1.6863e+03],
        [8.6666e-04, 2.0642e-01, 7.9951e+03, 2.7387e+03],
        [7.8526e-03, 9.2894e-02, 7.9951e+03, 1.7663e+03],
        [2.2246e-03, 9.2284e-02, 7.9951e+03, 2.0077e+03],
        [6.3988e-03, 6.1674e-02, 7.9951e+03, 1.0031e+03],
        [1.6870e-02, 6.8174e-02, 7.9951e+03, 1.3521e+03],
        [3.568

In [51]:
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.model=nn.Sequential(
            nn.Flatten(),
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 6),
        )

    def forward(self, x):
        z = self.model(x)
        return F.log_softmax(z, dim=1)

# Create model
model=SimpleModel().to('cpu')
# Create dataloader with sensible parameters
dl=DataLoader(ds,batch_size=4, num_workers=0, shuffle=True,drop_last=True)
# optimizer creation for training
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
model.train()
# training loop
for epoch in range(20):
    for batch, label in dl:
        # for each batch do a forward pass
        optimizer.zero_grad()
        oupt = model(batch)
        # calculate the loss
        loss_obj = F.nll_loss(oupt, label)
        # updates
        loss_obj.backward()
        optimizer.step()
#Do evaluation and the rest
model.eval()
# We simply evaluate on the training set, this is bad
# But also the results will be bad, we will make it better in the Full Example version
acc=Accuracy
# evaluation simple, quick to make tutorial fast
dl=DataLoader(ds,batch_size=240, num_workers=0, shuffle=False,drop_last=True)
for batch, label in dl:
    print("Result is bad, we will see why in a minute!")
    print(acc(torch.argmax(model(batch),dim=1),label))
    break